In [ ]:
!pip install -r requirements.txt

In [ ]:
# pip install python-docx aiogram langchain-community langchain-openai faiss-cpu

In [2]:
%%writefile /content/bot.py
import asyncio
from aiogram import Bot, Dispatcher, types, F
from aiogram.filters import Command
from aiogram.enums import ParseMode
from aiogram.client.default import DefaultBotProperties

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from docx import Document

from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

# Инициализация RAG
def initialize_rag():
  # загружаем документ
  doc = Document("/content/доработки Диадок.docx")
  text = "\n".join([paragraph.text for paragraph in doc.paragraphs])

  # Определяем сплиттер, делим текст на кусочки
  splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
  )
  split_documents = splitter.create_documents([text])

  # Если у вас нет видеокарты, укажите 'device': 'cpu'
  # Задаём embedding model
  hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cuda"}
  )

  # Создаём векторное хранилище
  db = FAISS.from_documents(
        split_documents, hf_embeddings_model
  )

  db.save_local("faiss_db")

  # Задаём ретривер
  # Ретривер - это такая надстройка над базой данных, которая может выдавать похожие документы, но не обязана их хранить.
  retriever = db.as_retriever()

  # Создаём простой шаблон
  template = """
    Отвечайте на вопрос, основываясь только на следующем контексте:

    {context}

    Question: {question}
  """
  # Создаём промпт из шаблона
  prompt = ChatPromptTemplate.from_template(template)

  # Укажите base_url и api_key для OpenRouter
  llm = ChatOpenAI(
    temperature=0.7,
    openai_api_key=OPENAI_API_KEY,  # Ваш API-ключ OpenRouter
    base_url="https://openrouter.ai/api/v1",
    model="openai/gpt-4o"
    # model="mistralai/mistral-7b-instruct"  # Укажите нужную модель
    , max_tokens=1000
    )


  # Объявляем функцию, которая будет собирать строку из полученных документов
  def format_docs(docs):
      return "\n\n".join([d.page_content for d in docs])


  # Создаём цепочку
  chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )

  return chain

# В ячейке Colab (лучше для безопасности)
TELEGRAM_BOT_TOKEN = input("Введите TELEGRAM_BOT_TOKEN: ")
OPENAI_API_KEY = input("Введите OPENAI_API_KEY: ")

# Инициализируем цепочку RAG при старте
chain = initialize_rag()

dp = Dispatcher()

@dp.message(Command('start'))
async def start_command(message: types.Message) -> None:
     await message.answer(f'<b>{message.from_user.full_name}</b>, добро пожаловать в бот помошник по данным, задай вопрос по теме!')

@dp.message()
async def echo_handler(message: types.Message) -> None:
    try:
        # Получаем ответ от цепочки RAG
        response = await chain.ainvoke(message.text)
        await message.reply(response)
    except Exception as e:
        await message.reply(f"Произошла ошибка: {str(e)}")


async def main() -> None:
    token = TELEGRAM_BOT_TOKEN
    bot = Bot(token
              ,default = DefaultBotProperties(parse_mode = ParseMode.HTML)
              )
    await dp.start_polling(bot)


if __name__ == "__main__":
    asyncio.run(main())

Writing /content/bot.py


In [ ]:
! python bot.py